# Day 2: 2.2

1. TF-IDF (tfidfVectorizer, cosineSimilarity)
2. Movie recommendation

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('03_tmdb_5000_movies.csv')
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [3]:
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [4]:
import json

In [5]:
json.loads(df.iloc[0,1])

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [6]:
def read_json(x):
    data = json.loads(x)
    name_list = []
    for i in data:
        name_list.append(i['name'].replace(" ", ""))
    return (' '.join(name_list).lower())

In [7]:
read_json(df.iloc[1,1])

'adventure fantasy action'

In [8]:
df.iloc[0,1]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [9]:
df.iloc[0,4]

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [10]:
df['BigString'] = df.apply(lambda x: read_json(x['genres']) + " " +read_json(x['keywords']), axis=1)

In [11]:
df.BigString[0]

'action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d'

In [12]:
model_df = df[['title', 'BigString']]

In [13]:
model_df.head()

,title,BigString
0,Avatar,action adventure fantasy sciencefiction cultur...
1,Pirates of the Caribbean: At World's End,adventure fantasy action ocean drugabuse exoti...
2,Spectre,action adventure crime spy basedonnovel secret...
3,The Dark Knight Rises,action crime drama thriller dccomics crimefigh...
4,John Carter,action adventure sciencefiction basedonnovel m...


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
matrix = TfidfVectorizer(lowercase=True,
                        stop_words='english',
                        min_df=0.01
                        )
X = matrix.fit_transform(df['BigString']).toarray()

In [16]:
X[0:5]

array([[0.50692921, 0.25165542, 0.29022759, 0.        , 0.        ,
        0.53302434, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.35370903,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.27670454, 0.33116438,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.     

In [17]:
matrix.get_feature_names()

['3d',
 'action',
 'adventure',
 'aftercreditsstinger',
 'airplane',
 'alien',
 'animation',
 'basedoncomicbook',
 'basedonnovel',
 'basedontruestory',
 'biography',
 'brotherbrotherrelationship',
 'comedy',
 'corruption',
 'crime',
 'daughter',
 'documentary',
 'drama',
 'drug',
 'duringcreditsstinger',
 'dyinganddeath',
 'dystopia',
 'escape',
 'family',
 'fantasy',
 'fathersonrelationship',
 'fbi',
 'femalenudity',
 'friends',
 'friendship',
 'highschool',
 'history',
 'horror',
 'hospital',
 'independentfilm',
 'investigation',
 'kidnapping',
 'londonengland',
 'losangeles',
 'love',
 'magic',
 'martialarts',
 'murder',
 'music',
 'musical',
 'mystery',
 'newyork',
 'nudity',
 'party',
 'police',
 'prison',
 'remake',
 'revenge',
 'romance',
 'sciencefiction',
 'sequel',
 'serialkiller',
 'sex',
 'sport',
 'suicide',
 'superhero',
 'survival',
 'suspense',
 'teenager',
 'thriller',
 'timetravel',
 'violence',
 'war',
 'wedding',
 'western',
 'womandirector',
 'worldwarii']

In [18]:
len(matrix.get_feature_names())

72

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

In NLP, Cosine similarity is a metric used to measure how similar the documents are irrespective of their size. Mathematically, it calculates the cosine of the angle between two vectors projected in a multi-dimensional space. 

The cosine similarity is advantageous because even if the two similar documents are far apart by the Euclidean distance because of the size (for example, the word “chatbot” could appear 50 times in one document and 10 times in another), they could still have a smaller angle between them. Smaller the angle, the higher the similarity.

https://www.engati.com/glossary/cosine-similarity#:~:text=In%20NLP%2C%20Cosine%20similarity%20is,in%20a%20multi%2Ddimensional%20space.

https://studymachinelearning.com/cosine-similarity-text-similarity-metric/

In [20]:
model_df.reset_index(inplace=True)
model_df.head()

,index,title,BigString
0,0,Avatar,action adventure fantasy sciencefiction cultur...
1,1,Pirates of the Caribbean: At World's End,adventure fantasy action ocean drugabuse exoti...
2,2,Spectre,action adventure crime spy basedonnovel secret...
3,3,The Dark Knight Rises,action crime drama thriller dccomics crimefigh...
4,4,John Carter,action adventure sciencefiction basedonnovel m...


In [21]:
similarity_matrix = cosine_similarity(X,X)

In [22]:
similarity_matrix[0]

array([1.        , 0.39549604, 0.17765044, ..., 0.10996193, 0.        ,
       0.        ])

In [23]:
def get_top_movies(title, n):
    idx = model_df[model_df.title == title]['index']
    sim = np.argsort(-1*(similarity_matrix[idx,:])).flatten()
    for i in sim[1:n+1]:
        print(model_df.iloc[i,1])

In [24]:
get_top_movies('Batman', 10)

Batman
Ghost Rider: Spirit of Vengeance
Batman & Robin
Hancock
The Specials
Hellboy
Zoom
Suicide Squad
Batman Forever
Spider-Man 2


In [25]:
get_top_movies('Avatar', 10)

Predator
Green Lantern
AVP: Alien vs. Predator
Clash of the Titans
Jupiter Ascending
Maleficent
Men in Black
Men in Black II
Zathura: A Space Adventure
I Am Number Four


In [26]:
get_top_movies('Spider-Man 2', 10)

Spider-Man 3
Superman IV: The Quest for Peace
Superman Returns
Superman III
The Toxic Avenger Part II
Superman II
Spider-Man
Batman
Ghost Rider: Spirit of Vengeance
Suicide Squad
